# Exploring the NASDAQOMX india indices

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

nasdaqOmx <- NasdaqOmx()

In [ ]:
indexMeta <- nasdaqOmx$Meta() %>%
    inner_join(nasdaqOmx$TimeSeries()) %>%
    group_by(ID, CODE, NAME) %>%
    filter(NAME %like% '% India %') %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    arrange(NAME) %>%
    collect()

indexMeta %>%
    print(n = Inf, width = Inf)

In [ ]:
#grab the data
startDate <- max(indexMeta$ST)
endDate <- min(indexMeta$ET)

nqRets <- NULL

for(id in indexMeta$ID){
    nqVals <- nasdaqOmx$TimeSeries() %>%
        filter(ID == id & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
        select(TIME_STAMP, CLOSE) %>%
        collect()
    
    nqRets <- merge.xts(nqRets, dailyReturn(xts(nqVals$CLOSE, nqVals$TIME_STAMP)))
}

names(nqRets) <- indexMeta$CODE

In [ ]:
#plot cumulative returns
Common.PlotCumReturns(nqRets, "NASDAQ India Focused Indices (TR)", "in USD")

In [ ]:
#returns in a table

retDf <- data.frame(CODE="", CUMULATIVE=0.0, ANNUALIZED=0.0)
for(j in 1:ncol(nqRets)){
    retDf <- rbind(retDf, c(toString(indexMeta$CODE[j]), 
                            100*as.numeric(Return.cumulative(nqRets[,j])), 
                            100*as.numeric(Return.annualized(nqRets[,j]))))
}
retDf <- retDf[-1,]
retDf[,2] <- round(as.numeric(retDf[,2]), 2)
retDf[,3] <- round(as.numeric(retDf[,3]), 2)

print(retDf)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)